# A Beginner's Guide to DSPy 🚀

Welcome to this tutorial on DSPy! DSPy is a framework from Stanford NLP that changes how we build applications with Large Language Models (LLMs). Instead of manually tweaking prompts, DSPy provides a structured, programming-based approach to prompt engineering.

**The Core Idea:** Separate the logic of your program from the parameters of your LLM. You define the steps your program should take, and DSPy's **optimizers** (called `Teleprompters`) will automatically figure out the best prompts to accomplish those steps.

This tutorial will cover the three fundamental concepts of DSPy:
1.  **Signatures**: Defining the input/output behavior of your LLM tasks.
2.  **Modules**: Building blocks that apply prompting techniques (like Chain of Thought) to Signatures.
3.  **Teleprompters**: Compilers that optimize your prompts based on data and a validation metric.

## 1. Setup and Configuration

First, let's install the necessary libraries. We'll also use `python-dotenv` to manage our API keys securely.

In [2]:
#%pip install dspy-ai python-dotenv openai -q

Next, create a file named `.env` in the same directory as this notebook and add your OpenAI API key like this:

```
OPENAI_API_KEY="your_api_key_here"
```

Now, let's load the key and configure DSPy to use a language model (in this case, GPT-3.5 Turbo).

In [5]:
import dspy
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

lm = dspy.LM('openai/gpt-3.5-turbo', api_key=openai_api_key)

# Apply the configuration
dspy.settings.configure(lm=lm)

## 2. Core Concepts: Signatures and Modules

### Signatures: Describing the Task
A **Signature** is a declarative specification of what you want an LLM to do. It defines the inputs and outputs of a task. Think of it as a function definition without the implementation.

The simplest way to define a signature is with a short string: `"input1, input2 -> output1, output2"`.

In [6]:
# Define a simple signature for a question-answering task
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often 1 to 5 words")

# You can also define this inline like so:
# simple_qa_signature = "question -> answer"

### Modules: Executing the Task
A **Module** is like a layer in a neural network. It takes one or more signatures and applies a prompting strategy to them. The most basic module is `dspy.Predict`, which creates a simple prompt from a signature. `dspy.ChainOfThought` is another module that asks the model to reason before answering.

In [10]:
# Create a module that uses our signature with a Chain of Thought strategy
generate_answer = dspy.ChainOfThought(BasicQA)

# Let's run it with an example question
question = "What is the capital of France?"
prediction = generate_answer(question=question)

print(f"Question: {question}")
print(f"Predicted Answer: {prediction.answer}")

# Print rationale if available in this DSPy version / LM response
rationale = None
for key in ("rationale", "reasoning", "chain_of_thought", "cot", "explanation"):
    try:
        val = getattr(prediction, key)
        if val:
            rationale = val
            break
    except AttributeError:
        continue

if rationale:
    print(f"Reasoning: {rationale}")
else:
    print("(No rationale available from ChainOfThought in this configuration)")

Question: What is the capital of France?
Predicted Answer: Paris
Reasoning: The capital of a country is typically the primary city where the government is located.


## 3. Optimization with Teleprompters

This is where DSPy truly shines. Instead of manually writing few-shot examples in your prompt, a **Teleprompter** can do it for you. It takes your program, a dataset of examples, and a validation metric, and then *compiles* your program into a more optimized version with better prompts.

### Step 1: Create a Small Training Set
DSPy learns from data. Let's create a few examples to teach our model.

In [11]:
# Create examples for training
train_data = [
    dspy.Example(question="What is the primary gas in Earth's atmosphere?", answer="Nitrogen").with_inputs("question"),
    dspy.Example(question="Which planet is known as the Red Planet?", answer="Mars").with_inputs("question"),
    dspy.Example(question="Who wrote 'Romeo and Juliet'?", answer="William Shakespeare").with_inputs("question"),
    dspy.Example(question="What is the largest ocean on Earth?", answer="Pacific Ocean").with_inputs("question")
]

### Step 2: Define a Validation Metric
The optimizer needs a way to know if its changes are good or bad. We define a simple function that checks if the predicted answer matches the true answer.

In [12]:
# Validation logic: true if predicted answer contains the gold answer (case-insensitive)
def validate_answer(example, pred, trace=None):
    return example.answer.lower() in pred.answer.lower()


### Step 3: Compile the Program
We'll use the `BootstrapFewShot` teleprompter. It will create few-shot examples from the `train_data` and add them to the prompt of our `generate_answer` module.

In [13]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=2) # We'll use 2 few-shot examples
teleprompter = BootstrapFewShot(metric=validate_answer, **config)

# Compile the program
optimized_program = teleprompter.compile(generate_answer, trainset=train_data)


 50%|█████     | 2/4 [00:02<00:02,  1.47s/it]

Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.


### Step 4: Use the Optimized Program
Now we can use our `optimized_program` just like the original one, but it will perform better because it has learned effective few-shot examples to include in its prompt.

In [14]:
new_question = "What is the speed of light?"
prediction = optimized_program(question=new_question)

print(f"Question: {new_question}")
print(f"Predicted Answer: {prediction.answer}")

# Optionally inspect the last prompt sent to the LLM, if available
prompt_repr = None
try:
    # Some versions expose an inspector or history
    if hasattr(lm, 'history') and lm.history:
        prompt_repr = lm.history[-1].get('prompt') or lm.history[-1]
except Exception:
    pass

if prompt_repr is not None:
    print("\n--- Last Prompt (repr) ---\n", prompt_repr)
else:
    print("\n(No prompt history available from the configured LM)")

Question: What is the speed of light?
Predicted Answer: 299,792 km/s

--- Last Prompt (repr) ---
 {'prompt': None, 'messages': [{'role': 'system', 'content': 'Your input fields are:\n1. `question` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `answer` (str): often 1 to 5 words\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## answer ## ]]\n{answer}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Answer questions with short factoid answers.'}, {'role': 'user', 'content': 'This is an example of the task, though some input or output fields are not supplied.\n\n[[ ## question ## ]]\nWhat is the largest ocean on Earth?'}, {'role': 'assistant', 'content': '[[ ## reasoning ## ]]\nNot supplied for this particular example. \n\n[[ ## answer ## ]]\nPacific Ocean\n\n[[ ## completed ## ]]\n'}, {'role': 'user', 'content

## 4. Code Generation Example

DSPy can help you structure code generation tasks with clear input/output signatures and reusable prompting strategies. Below is a simple example that turns a natural-language description into Python code. We’ll also ask the model to briefly explain the solution.

In [ ]:
# Define a signature for code generation with a short explanation
class CodeGen(dspy.Signature):
    """Generate Python code to solve a small task, and include a short explanation."""
    task = dspy.InputField(desc="A natural language description of the coding task.")
    code = dspy.OutputField(desc="Return only valid Python code (no markdown/code fences).")
    explanation = dspy.OutputField(desc="1-2 sentences explaining the approach.")

# Use Chain of Thought to encourage planning before coding
codegen = dspy.ChainOfThought(CodeGen)

# Try it on a simple task
spec = "Write a Python function fibonacci(n) that returns a list of the first n Fibonacci numbers."
result = codegen(task=spec)

# Helper to strip Markdown code fences if the model included them
def extract_code(text: str) -> str:
    import re
    if not text:
        return ""
    # Prefer content between triple backticks
    m = re.search(r"```(?:[a-zA-Z0-9_+\-\.]+)?\n(.*?)```", text, re.DOTALL)
    if m:
        return m.group(1).strip()
    stripped = text.strip()
    if stripped.startswith("```") and stripped.endswith("```"):
        # Remove surrounding backticks and optional language tag on first line
        inner = stripped.strip("`")
        lines = inner.splitlines()
        if lines and lines[0].strip().lower().startswith("python"):
            lines = lines[1:]
        return "\n".join(lines).strip()
    return stripped

sanitized_code = extract_code(result.code)

print("Task:\n", spec)
print("\nGenerated Code:\n", sanitized_code)
print("\nExplanation:\n", result.explanation)

# Execute the generated code safely in a sandboxed namespace
ns = {}
try:
    compiled = compile(sanitized_code, filename="<generated>", mode="exec")
    exec(compiled, ns)
    if "fibonacci" in ns and callable(ns["fibonacci"]):
        print("\nSanity Check:", ns["fibonacci"](7))
except SyntaxError as e:
    print("\nExecution error: SyntaxError", f"line {e.lineno} col {e.offset}:", e.text or "", "|", e.msg)
except Exception as e:
    print("\nExecution error:", type(e).__name__, str(e))

Task:
 Write a Python function fibonacci(n) that returns a list of the first n Fibonacci numbers.

Generated Code:
 def fibonacci(n):
    fib_list = []
    a, b = 0, 1
    for _ in range(n):
        fib_list.append(a)
        a, b = b, a + b
    return fib_list

Explanation:
 The function `fibonacci(n)` generates the first n Fibonacci numbers by iteratively calculating each number based on the sum of the previous two numbers.

Sanity Check: [0, 1, 1, 2, 3, 5, 8]


## Conclusion

Congratulations! You've just learned the fundamentals of DSPy. You've seen how to:

1. Define behavior with `Signatures`.
2. Encapsulate logic and strategies with `Modules` (e.g., `ChainOfThought`).
3. Automatically optimize prompts with `Teleprompters` (e.g., `BootstrapFewShot`).
4. Apply these concepts to code generation: turning natural-language specs into runnable Python code with brief explanations.

This programmatic approach makes building reliable LLM applications much more systematic. From here, you can explore more advanced teleprompters, other modules (e.g., `ReAct`, `Refine`, `BestOfN`), add evaluation metrics, or integrate retrieval tools later if your application needs grounding.